# Review Sentiment Analysis for Product Analysis

In [1]:
import re
import string

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf

from keras import layers
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Dataset

The dataset used is 'Flipkart Product reviews with sentiment Dataset'. It is publicly available to download at URL : https://www.kaggle.com/datasets/niraliivaghani/flipkart-product-customer-reviews-dataset


In our case, the database is already present at the project's github repository inside 'data' folder.

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/waleashish/online-product-review-classification/main/data/Dataset-SA.csv",
                 low_memory=False)

The online dataset contains many columns such as 'Product Name', 'Product Price', etc. We only need 'Sentiment' and 'Summary' for our modelling.

In [3]:
dataset = df[['Summary', 'Sentiment']]

In [4]:
# This block contains all the helper functions

"""
    This function performs clearing process on the input text.
    Arguments:
      text : sentence to be processed
    Returns:
      The clean sentence
"""
def process_sentence(text):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove unwanted text patterns (hyperlinks, hashtags, etc.)
    text = re.sub(r'\$\w*', '', str(text))
    # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', str(text))
    # remove hyperlinks
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', str(text))
    # remove hashtags
    # only removing the hash # sign from the word
    text = re.sub(r'#', '', str(text))
    # tokenize tweets
    text_tokens = word_tokenize(text)
    # make all word tokens lower case
    text_tokens = [token.lower() for token in text_tokens]
    # remove punctuation
    text_clean = ""
    for word in text_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            text_clean += stemmer.stem(word) + " "

    return text_clean[:-1]

> Our 'Sentiment' column has sentiments as strings. We need to encode them into integers. We'll use sklearn's `LabelEncoder` for this purpose.

> Our 'Summary' column, which contains the reviews, has original reviews. We need to preprocess them to remove punctuations, stopwords, etc.

In [ ]:
le = LabelEncoder()
# Encode the sentiments as integers
dataset['label'] = le.fit_transform(df['Sentiment'])
# Pre process the reviews
dataset['Summary'] = dataset['Summary'].apply(lambda x : process_sentence(x))

Create the training, testing and validation datasets.

In [6]:
# Create training and testing data
train_sentences, temp_test_sentences, train_labels, temp_test_labels = train_test_split(dataset['Summary'],
                                                                                        dataset['label'],
                                                                                        test_size=0.2)

# Create validation and testing data
valid_sentences, test_sentences, valid_labels, test_labels = train_test_split(temp_test_sentences,
                                                                              temp_test_labels,
                                                                              test_size=0.5)

## Model

We are creating following model architecture:


1.   Text Vectorization Layer
2.   Embedding Layer
3.   LSTM Layers
4.   Dense Layers
5    Output Dense Layer


Our compilation parameters:

1.   Optimizer: Adam
2.   Loss: Sparse Categorical Cross Entropy
3.   Metric: Accuracy


Our Fitting parameters:

1.   Epochs: 10



In [32]:
# Declare variables
MAX_VOCAB_SIZE = 10000
MAXLEN = 25

# Create text vectorization layers
text_vectorizer = layers.TextVectorization(max_tokens=MAX_VOCAB_SIZE,
                                           output_mode='int',
                                           output_sequence_length=MAXLEN)
text_vectorizer.adapt(train_sentences)
# Create embedding layer
embedding = layers.Embedding(input_dim=MAX_VOCAB_SIZE,
                             output_dim=128,
                             input_length=MAXLEN)

In [34]:
# Create model
inputs = tf.keras.layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(3, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs, name="review_sentiment_analysis")

# Compile the model
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# Fit the model
history = model.fit(x=train_sentences,
                    y=train_labels,
                    epochs=10,
                    validation_data=(valid_sentences, valid_labels))

Epoch 1/10
5127/5127 [==============================] - 70s 13ms/step - loss: 0.2927 - accuracy: 0.9072 - val_loss: 0.2535 - val_accuracy: 0.9213
Epoch 2/10
5127/5127 [==============================] - 44s 9ms/step - loss: 0.2375 - accuracy: 0.9237 - val_loss: 0.2377 - val_accuracy: 0.9239
Epoch 3/10
5127/5127 [==============================] - 44s 8ms/step - loss: 0.2137 - accuracy: 0.9301 - val_loss: 0.2333 - val_accuracy: 0.9232
Epoch 4/10
5127/5127 [==============================] - 43s 8ms/step - loss: 0.1973 - accuracy: 0.9343 - val_loss: 0.2416 - val_accuracy: 0.9234
Epoch 5/10
5127/5127 [==============================] - 43s 8ms/step - loss: 0.1841 - accuracy: 0.9393 - val_loss: 0.2486 - val_accuracy: 0.9205
Epoch 6/10
5127/5127 [==============================] - 44s 9ms/step - loss: 0.1722 - accuracy: 0.9433 - val_loss: 0.2567 - val_accuracy: 0.9211
Epoch 7/10
5127/5127 [==============================] - 42s 8ms/step - loss: 0.1626 - accuracy: 0.9462 - val_loss: 0.2666 - val_a

## Testing evaluation

In [35]:
model.evaluate(test_sentences, test_labels)

641/641 [==============================] - 3s 4ms/step - loss: 0.3136 - accuracy: 0.9148


[0.31364279985427856, 0.9148054122924805]

## Prediction on new data

In [36]:
def predict_sentiment(model, sentences):
    # Get prediction probabilities
    pred_probs = model.predict(sentences)

    # Get the index of maximum probability, that is our label
    label = np.argmax(pred_probs, axis=-1)

    # Convert the label back to sentiment and return it
    return list(le.inverse_transform(label))

In [39]:
sent = [
    "Wow!!! Its the very first word that i said as i put on this headset. Amazing Product, and Delivery time was also great.",
    "Please take back product because of not working properly.",
    "An amazing experience. Never seen never dreamt experiences. A worthy entertainer. Cost could have been more affordable.",
    "It was a pretty decent set up & fun to play games on as well.",
    "Useless item!"
    ]

predict_sentiment(model, sent)

1/1 [==============================] - 0s 36ms/step


['positive', 'negative', 'positive', 'positive', 'negative']